In [ ]:
import pandas as pd
import numpy as np
import dash
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output

### Explatory Data Analysis with Dashboard for Supermarket Sales Dataset


The main aim of this project is to analyse the Supermarket Sales dataset to create an interactive analytical dashboard that analyses the performance of the supermarket's overall activity together with individual branches. 

The analysis will perform explanatory data analysis based on:
   >1. Total weekly and daily sales, cost of goods and gross profit for the supermarket combined and individual branches.
   >2. Customer types
   >3. Product line and rating.  

The dataset has the following features:
* Invoice id: Computer generated sales slip invoice identification number
* Branch: Branch of supercenter (3 branches are available identified by A, B and C).
* City: Location of supercenters
* Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
* Gender: Gender type of customer
* Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
* Unit price: Price of each product in dollars
* Quantity: Number of products purchased by customer
* Tax 5%: tax fee for customer buying
* Total: Total price including tax
* Date: Date of purchase (Record available from January 2019 to March 2019)
* Time: Purchase time (10am to 9pm)
* Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
* COGS: Cost of goods sold
* Gross margin percentage: Gross margin percentage
* Gross income: Gross income
* Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)
  
The dataset can be found [Here](https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales/code)

In [ ]:
#create a data frame

df=pd.read_csv('Sales Dataset _with Correction.csv')
df

In [ ]:
#check data type
df.dtypes

In [ ]:
#change the date format to dt
df['Date']=pd.to_datetime(df['Date'])
df

In [ ]:
# get the month name
df['Month'] = df['Date'].dt.month_name()
df['Day'] = df['Date'].dt.day_name()
df

In [ ]:
# get the week number
df['Week'] = df['Date'].dt.isocalendar().week
df

In [ ]:
df['Time'] = pd.to_datetime(df.Time)

In [ ]:
df.dtypes

In [ ]:
# find all the categories in the dataset

for col in df.columns:
    if df[col].dtype == object and col!='Invoice ID':
        print(f'The categories for {col} is: {df[col].unique()}')
        print("")

#### Aggregate the braches sales

In [ ]:
#Branch A
a_branch = df[df['Branch']== 'A']
a_branch

In [ ]:
#get branch b
b_branch = df[df['Branch']== 'B']
b_branch

In [ ]:
#get branch c
c_branch = df[df['Branch']== 'C']
c_branch

### Group the branche's customer based on customer type

In [ ]:
#get branch members
a_branch_members = a_branch[a_branch['Customer type']=='Member']
a_branch_members

In [ ]:
#get the non-members
a_branch_normal = a_branch[a_branch['Customer type']=='Normal']
a_branch_normal

In [ ]:
min_per = df['Date'].min()
max_per = df['Date'].max()
min_per, max_per

In [ ]:
#Total figures df
total_sales = round(df['Total'].sum(),2)
gross_profit = round(df['gross profit'].sum(),2)
cog = round(df['COGS'].sum(),2)
average_gross_per = round(df['gross margin percentage(unit)'].mean(),2)
cust_rate = round(df['Rating'].mean(),2)


In [ ]:
#Total figures branch A
A_total_sales = round(a_branch['Total'].sum(),2)
A_gross_profit = round(a_branch['gross profit'].sum(),2)
A_average_gross_per = round(a_branch['gross margin percentage(unit)'].mean(),2)
A_cog = round(a_branch['COGS'].sum(),2)
A_cust_rate = round(a_branch['Rating'].mean(),2)
A_total_sales

In [ ]:
#Total figures branch B
B_total_sales = round(b_branch['Total'].sum(),2)
B_gross_profit = round(b_branch['gross profit'].sum(),2)
B_average_gross_per = round(b_branch['gross margin percentage(unit)'].mean(),2)
B_cog = round(b_branch['COGS'].sum(),2)
B_cust_rate = round(b_branch['Rating'].mean(),2)

In [ ]:
#Total figures branch C
C_total_sales = round(c_branch['Total'].sum(),2)
C_gross_profit = round(c_branch['gross profit'].sum(),2)
C_average_gross_per = round(c_branch['gross margin percentage(unit)'].mean(),2)
C_cog = round(c_branch['COGS'].sum(),2)
C_cust_rate = round(c_branch['Rating'].mean(),2)

In [ ]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])             

In [ ]:
app.layout = html.Div([
    dbc.Row([html.H1('Supermarket Sales Dashboard.'),
            html.A('Link to the dataset', href='https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales/code', target='_blank')

            ], style={'background-color':'#80cdc1', 'margin':'5px','border-radius':'10px', 'font-size':'12px'}),
    
    
    dbc.Row([
        dbc.Col([html.H1('Reporting\nPeriod: 01/01/2019-30/03/2019',style={'font-size':'16px', 'padding-top':'20px'})],style={'background-color':'#dfc27d', 'margin':'10px','border-radius':'25px'}),
        dbc.Col([html.H1(id='total-sales', style={'font-size':'16px', 'padding-top':'20px'})],style={'background-color':'#dfc27d','margin':'10px','border-radius':'25px'}),
        dbc.Col([html.H1(id='gross_profit',style={'font-size':'16px', 'padding-top':'20px'})],style={'background-color':'#dfc27d','margin':'10px','border-radius':'25px'}),
        dbc.Col([html.H1(id='average_gross%',style={'font-size':'16px', 'padding-top':'20px'})],style={'background-color':'#dfc27d','margin':'10px','border-radius':'25px'}),
        dbc.Col([html.H1(id='cog', style={'font-size':'16px', 'padding-top':'20px'})],style={'background-color':'#dfc27d','margin':'10px','border-radius':'25px'}),
        dbc.Col([html.H1(id='cust_rate', style={'font-size':'16px', 'padding-top':'20px'})],style={'background-color':'#dfc27d','margin':'10px','border-radius':'25px'})
        
    ]),
    
        dbc.Row([dcc.RadioItems(
            id='supermarket',
            options=[{'label':'Combined Total', 'value':'df'},
                 {'label': 'Branch A', 'value': 'a_branch'},
                 {'label': 'Branch B', 'value': 'b_branch'},
                 {'label': 'Branch C', 'value': 'c_branch'}               
                ], 
            value='df',
            labelStyle={'display': 'in-line'}),
            
             
            ],style={'margin':'5px'}),
    dbc.Row([dbc.Col([dcc.Graph(id='sales_trend', figure={})], width=6),
             dbc.Col([dcc.Graph(id='avarage_sales', figure={})], width=6)
            ]),
    
    dbc.Row([dbc.Col([dcc.Graph(id='product_per_analysis', figure={}, config={'displayModeBar':False})
                     ]),
             dbc.Col([dcc.Graph(id='pyment_method', figure={}, config={'displayModeBar': False}),
                 
             ])
        
    ])
       
    
], style={'margin':'10px', 'back-ground':'#ffffcc'})

In [ ]:
@app.callback(
    Output('total-sales', 'children'),
    [Input('supermarket', 'value')]
)

def update_output(user_select):
    if user_select == 'df':
        return f'Total Sales: {total_sales:,}'
    elif user_select == 'a_branch':
        return f'Total Sales: {A_total_sales:,}'
    elif user_select == 'b_branch':
        return f'Total Sales: {B_total_sales:,}'
    elif user_select == 'c_branch':
        return f'Total Sales: {C_total_sales:,}'
####################################################

@app.callback(
    Output('gross_profit', 'children'),
    [Input('supermarket', 'value')]
)

def update_output_g(user_select_g):
    if user_select_g == 'df':
        return f'Gross Profit: {gross_profit:,}'
    elif user_select_g == 'a_branch':
        return f'Gross Profit: {A_gross_profit:,}'
    elif user_select_g == 'b_branch':
        return f'Gross Profit: {B_gross_profit:,}'
    elif user_select_g == 'c_branch':
        return f'Gross Profit: {C_gross_profit:,}'
    
####################################################

@app.callback(
    Output('average_gross%', 'children'),
    [Input('supermarket', 'value')]
)

def update_output_(user_select_):
    if user_select_ == 'df':
        return f'Gross Profit %: {average_gross_per:,}'
    elif user_select_ == 'a_branch':
        return f'Gross Profit %: {A_average_gross_per:,}'
    elif user_select_ == 'b_branch':
        return f'Gross Profit %: {B_average_gross_per:,}'
    elif user_select_ == 'c_branch':
        return f'Gross Profit %: {C_average_gross_per:,}'
####################################################

@app.callback(
    Output('cog', 'children'),
    [Input('supermarket', 'value')]
)

def update_output_c(user_select_c):
    if user_select_c == 'df':
        return f'Cost of Goods: {cog:,}'
    elif user_select_c == 'a_branch':
        return f'Cost of Goods: {A_cog:,}'
    elif user_select_c == 'b_branch':
        return f'Cost of Goods: {B_cog:,}'
    elif user_select_c == 'c_branch':
        return f'Cost of Goods: {C_cog:,}'
    
#####################################################
@app.callback(
    Output('cust_rate', 'children'),
    [Input('supermarket', 'value')]
)

def update_output_r(user_select_r):
    if user_select_r == 'df':
        return f'Average Customer Rating: {cust_rate:,}'
    elif user_select_r == 'a_branch':
        return f'Average Customer Rating: {A_cust_rate:,}'
    elif user_select_r == 'b_branch':
        return f'Average Customer Rating: {B_cust_rate:,}'
    elif user_select_r == 'c_branch':
        return f'Average Customer Rating: {C_cust_rate:,}'
    


In [ ]:
#Sales line graph

@app.callback(
    [Output('sales_trend', 'figure')],
    [Input('supermarket', 'value')]

)

def update_sales_graph(user_select):
    
    if user_select == 'df':
        df_grouped = df.groupby('Week').sum(numeric_only=True).round(2) 
        fig = px.line(df_grouped, x= df_grouped.index, y=df_grouped['Total'], title=f'Combined Weekly Sales.',markers='o')
        fig.update_traces(line_color='#a6611a')
        fig.add_annotation(x=10, y=17300, text='Week 8 is <br> a week that<br> the supermarket has<br> recorded the least sales.', showarrow=False)
        fig.update_layout(xaxis={'showgrid':False})
        
        return(fig,)
    
    elif user_select == 'a_branch':
        A_grouped = a_branch.groupby('Week').sum(numeric_only=True).round(2) 
        A_grouped_min = A_grouped['Total'].min()
        A_grouped_min_ind = A_grouped.idxmin()
        
        fig = px.line(A_grouped, x= A_grouped.index, y=A_grouped['Total'], title=f'Branch A Weekly Sales.',markers='o')
        fig.update_traces(line_color='#a6611a')
        fig.add_annotation(x=11, y=5536.83, text='Week 8 is <br> a week that<br> the Branch A has<br> recorded the least sales.', showarrow=True, arrowsize=12)
        fig.update_layout(xaxis={'showgrid':False})
        return (fig,)
    
    elif user_select == 'b_branch':
        B_grouped = b_branch.groupby('Week').sum(numeric_only=True).round(2)
        fig = px.line(B_grouped, x= B_grouped.index, y=B_grouped['Total'],title=f'Branch B Weekly Sales.',markers='o')
        fig.update_traces(line_color='#a6611a')
        fig.add_annotation(x=10, y=5812, text='Week 8 is <br> a week that<br> the Branch B has<br> recorded the least sales.', showarrow=True, arrowsize=12)
        fig.update_layout(xaxis={'showgrid':False})
        
        return(fig,)
    
    elif user_select == 'c_branch':
        C_grouped = c_branch.groupby('Week').sum(numeric_only=True).round(2)
        fig = px.line(C_grouped, x= C_grouped.index, y=C_grouped['Total'],title=f'Branch C Weekly Sales.',markers='o' )
        fig.update_traces(line_color='#a6611a')
        fig.add_annotation(x=11, y=5979, text='Week 8 is <br> a week that<br> the Branch C has<br> recorded the least sales.', showarrow=True, arrowsize=12)
        fig.update_layout(xaxis={'showgrid':False})
        
        return(fig,) 

In [ ]:
#average sales

@app.callback(
    [Output('avarage_sales', 'figure')],
    [Input('supermarket', 'value')]

)


def update_sales_graph(user_select):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    if user_select == 'df':
        df_grouped = df.groupby('Day').mean(numeric_only=True).reindex(days)
        fig = px.line(df_grouped, x= df_grouped.index, y=df_grouped['Total'], title=f'Combined Daily Sales.', markers='o')
        fig.update_traces(line_color='#a6611a')
        fig.add_annotation(x='Friday', y=310, text='Saturday is <br> the maximum sales recorded.<br> the supermarket has<br> the least sales recorded on Monday.', showarrow=False)
        
        fig.update_layout(xaxis={'showgrid':False})
        
        return(fig,)
    
    elif user_select == 'a_branch':
        A_grouped = a_branch.groupby('Day').sum(numeric_only=True).reindex(days)
        
        fig = px.line(A_grouped, x= A_grouped.index, y=A_grouped['Total'], title=f'Branch A Daily Sales.',markers='o')
        fig.update_traces(line_color='#a6611a')
        fig.add_annotation(x='Friday', y=14000, text='Sunday has the<br> maximum sales recorded.<br> the supermarket has<br> the least sales recorded on Wednesday.', showarrow=False)
        fig.update_layout(xaxis={'showgrid':False})
        return (fig,)
    
    elif user_select == 'b_branch':
        B_grouped = b_branch.groupby('Day').sum(numeric_only=True).reindex(days) 
        fig = px.line(B_grouped, x= B_grouped.index, y=B_grouped['Total'],title=f'Branch B Daily Sales.',markers='o')
        fig.update_traces(line_color='#a6611a')
        fig.add_annotation(x='Thursday', y=12000, text='While Saturday has<br> the maximum sales<br> Sunday has the minimum sales.', showarrow=False)
        fig.update_layout(xaxis={'showgrid':False})
        
        return(fig,)
    
    elif user_select == 'c_branch':
        C_grouped = c_branch.groupby('Day').sum(numeric_only=True).reindex(days) 
        fig = px.line(C_grouped, x= C_grouped.index, y=C_grouped['Total'],title=f'Branch C Daily Sales.',markers='o' )
        fig.update_traces(line_color='#a6611a')
        fig.update_layout(xaxis={'showgrid':False})
        fig.add_annotation(x='Wednesday', y=12000, text='Tuseday has<br> the minimum sales<br> while Saturday has the maximum sales.', showarrow=False)
        
        
        return(fig,) 


       

In [ ]:
# graph product_per_analysis

@app.callback(
    [Output('product_per_analysis', 'figure')],
    [Input('supermarket', 'value')]

)

def update_sales_graph(user_select):
    
    if user_select == 'df':
        df_perf = df.groupby('Product line').sum(numeric_only=True)
        fig = px.bar(df_perf, x= df_perf.index, y=df_perf['Total'],color=df_perf.index,
                     color_discrete_map={'Electronic accessories':'#01665e', 'Fashion accessories':'#d8b365',
                                        'Food and beverages':'#8c510a','Health and beauty': '#01665e',
                                        'Home and lifestyle':'#c7eae5', 'Sports and travel':'#f6e8c3'},
                                         title=f'Combined Product Line Sales.')
       
        
        return(fig,)
    
    elif user_select == 'a_branch':
        
        A_pref = a_branch.groupby('Product line').sum(numeric_only=True) 
        fig = px.bar(A_pref, x= A_pref.index,
                     y=A_pref['Total'],
                     color=A_pref.index,
                     color_discrete_map={'Electronic accessories':'#01665e', 'Fashion accessories':'#d8b365',
                                        'Food and beverages':'#8c510a','Health and beauty': '#01665e',
                                        'Home and lifestyle':'#c7eae5', 'Sports and travel':'#f6e8c3'},
                                        title=f'Branch A Product Line Sales.')
      
        
        return (fig,)
    
    elif user_select == 'b_branch':
        B_pref = b_branch.groupby('Product line').sum(numeric_only=True) 
        fig = px.bar(B_pref, x= B_pref.index, 
                     y=B_pref['Total'],
                     color=B_pref.index,
                     color_discrete_map={'Electronic accessories':'#01665e', 'Fashion accessories':'#d8b365',
                                        'Food and beverages':'#8c510a','Health and beauty': '#01665e',
                                        'Home and lifestyle':'#c7eae5', 'Sports and travel':'#f6e8c3'},
                     title=f'Branch B Product Line Sales.')
        
        return(fig,)
    
    elif user_select == 'c_branch':
        C_pref = c_branch.groupby('Product line').sum(numeric_only=True) 
        fig = px.bar(C_pref, x= C_pref.index, 
                     y=C_pref['Total'],
                     color=C_pref.index,
                     color_discrete_map={'Electronic accessories':'#01665e', 'Fashion accessories':'#d8b365',
                                        'Food and beverages':'#8c510a','Health and beauty': '#01665e',
                                        'Home and lifestyle':'#c7eae5', 'Sports and travel':'#f6e8c3'},
                     title=f'Branch C Product Line Sales.' )
       
        return(fig,) 

In [ ]:
# graph member and payment type

@app.callback(
    [Output('pyment_method', 'figure')],
    [Input('supermarket', 'value')]

)

def update_sales_graph(user_select,):
    
    if user_select == 'df':
        fig = px.sunburst(df, path=['Customer type', 'Payment', 'Gender'], values='Total', color='Customer type', color_discrete_sequence=px.colors.qualitative.Pastel, title='Customer Segmentation by Membership, Payment Type and Gender')
        return(fig,)
    
    elif user_select == 'a_branch':
        
        fig = px.sunburst(a_branch, path=['Customer type', 'Payment','Gender'], values='Total', color='Customer type', color_discrete_sequence=px.colors.qualitative.Pastel,title='Customer Segmentation by Membership, Payment Type and Gender')
        return (fig,)
    
    elif user_select == 'b_branch':
        
        fig = px.sunburst(b_branch, path=['Customer type', 'Payment', 'Gender'], values='Total', color='Customer type', color_discrete_sequence=px.colors.qualitative.Pastel,title='Customer Segmentation by Membership, Payment Type and Gender')
        return(fig,)
    
    elif user_select == 'c_branch':
        
        fig = px.sunburst(c_branch, path=['Customer type', 'Payment', 'Gender'], values='Total', color='Customer type',color_discrete_sequence=px.colors.qualitative.Pastel,title='Customer Segmentation by Membership, Payment Type and Gender')
        return(fig,) 

In [247]:
if __name__ == '__main__':
    app.run(debug=False, jupyter_mode='inline')